# Fundamental Indicators

### *Libraries*

In [9]:
# System
import os, time, sys, glob
from pathlib import Path
from datetime import date, datetime, timedelta

In [232]:
# Data
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3

In [17]:
# APIs
from dotenv import load_dotenv
import finnhub as fh

In [18]:
# Visualization
import panel as pn
import panel.widgets as pnw

import plotly.express as px
import plotly.graph_objects as go
pn.extension('plotly')
pn.extension()

import holoviews as hv
from holoviews import opts
import hvplot.pandas

import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
# Fundamental Analysis
from fundamentals import Fundamental

---

### *Data*

#### *API Data Methods*

##### FinnHub API

In [20]:
# load API keys
api_key_path = Path('../resources/api_keys.env')  # replace with `/path/to/your/api_keys.env`
load_dotenv(api_key_path)

# Set FinnHub API key
finnhub_api_key = os.getenv("FINNHUB_API_KEY")

In [21]:
print(f"FINNHUB_API_KEY: {type(finnhub_api_key)}")

FINNHUB_API_KEY: <class 'str'>


In [23]:
# Create FinnHub API object
finnhub_client = fh.Client(api_key=finnhub_api_key)

# Method to obtain json data from FinnHub
def finnhub_data(ticker):
    
    data = finnhub_client.company_basic_financials(ticker, "")
    data_df = pd.DataFrame(data)
    time.sleep(1)
    
    return data_df

##### FinnHub API data parsing methods

In [147]:
# Get stock fundamental data from FinnHub
def fundamentals_finnhub(tickers : list) -> DataFrame:
    '''Returns pd.DataFrame with fundamentals of tickers within tickers_df
    
    ...
    Parameters
    ----------
    tickers_df : pd.DataFrame - contains tickers for given index or sector under 
        the "Symbol" column which is the DataFrame key
        
    Returns
    -------
    result_df : pd.DataFrame - securities fundamental data
    '''
    
    def get_data(ticker): 
        data = finnhub_data(ticker)['metric'].transpose()
        return data
    
    fundamentals_df = DataFrame()
    fundamentals_df[tickers[0]] = get_data(tickers[0])
    
    for ticker in tickers[1:]:
        fundamentals_df[ticker] = get_data(ticker)
    
    return fundamentals_df.transpose()

In [148]:
fundamentals = fundamentals_finnhub(['FB','AAPL','AMZN'])
fundamentals

,10DayAverageTradingVolume,13WeekPriceReturnDaily,26WeekPriceReturnDaily,3MonthAverageTradingVolume,52WeekHigh,52WeekHighDate,52WeekLow,52WeekLowDate,52WeekPriceReturnDaily,5DayPriceReturnDaily,...,roiAnnual,roiTTM,tangibleBookValuePerShareAnnual,tangibleBookValuePerShareQuarterly,tbvCagr5Y,totalDebt/totalEquityAnnual,totalDebt/totalEquityQuarterly,totalDebtCagr5Y,yearToDatePriceReturnDaily,annual
FB,20.54705,12.89619,10.97135,425.59724,331.81,2021-04-29,200.69,2020-05-14,48.83471,-4.61221,...,22.19312,24.61803,38.12461,40.16051,36.47031,0.40767,0.39429,10.67212,11.75135,NaN
AAPL,96.79679,-5.85063,6.86735,2185.98209,145.09,2021-01-25,75.0525,2020-05-15,64.696,-2.11965,...,25.44284,33.53747,3.84873,4.1458,-9.95011,173.0926,175.8435,11.94642,-3.94905,NaN
AMZN,4.62609,-1.6722,3.00721,78.17968,3554,2021-04-30,2330,2020-05-27,34.91429,-2.08743,...,12.83085,15.34307,145.9364,174.8016,53.4313,54.63471,66.93864,24.19046,-1.04485,NaN
